In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

# Naval Propulsion Dataset

In [2]:
naval_data = pd.read_csv('naval.csv')

In [3]:
naval_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Data columns (total 18 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   1 - Lever position (lp) [ ]                         11934 non-null  float64
 1   2 - Ship speed (v) [knots]                          11934 non-null  float64
 2   3 - Gas Turbine shaft torque (GTT) [kN m]           11934 non-null  float64
 3   4 - Gas Turbine rate of revolutions (GTn) [rpm]     11934 non-null  float64
 4   5 - Gas Generator rate of revolutions (GGn) [rpm]   11934 non-null  float64
 5   6 - Starboard Propeller Torque (Ts) [kN]            11934 non-null  float64
 6   7 - Port Propeller Torque (Tp) [kN]                 11934 non-null  float64
 7   8 - HP Turbine exit temperature (T48) [C]           11934 non-null  float64
 8   9 - GT Compressor inlet air temperature (T1) [C]    11934 non-null  float64


In [4]:
y = naval_data[naval_data.columns[16]]
X = naval_data.iloc[:,0:16]
y.info()
print(y)
X.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11934 entries, 0 to 11933
Series name: 17 - GT Compressor decay state coefficient.
Non-Null Count  Dtype  
--------------  -----  
11934 non-null  float64
dtypes: float64(1)
memory usage: 93.4 KB
0        0.95
1        0.95
2        0.95
3        0.95
4        0.95
         ... 
11929    1.00
11930    1.00
11931    1.00
11932    1.00
11933    1.00
Name: 17 - GT Compressor decay state coefficient., Length: 11934, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Data columns (total 16 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   1 - Lever position (lp) [ ]                         11934 non-null  float64
 1   2 - Ship speed (v) [knots]                          11934 non-null  float64
 2   3 - Gas Turbine shaft torque (GTT) [kN m]           11934 non-null  float64
 3   4 

In [5]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = y.to_numpy()
y = y.reshape(-1,1)
y = scaler_y.fit_transform(y)

print(X)
print(y)

[[-1.53324812 -1.54919334 -1.2166697  ... -0.9486833  -1.02557391
  -1.14440517]
 [-1.17161304 -1.161895   -0.91565197 ... -0.9486833  -0.88780617
  -0.7403042 ]
 [-0.77191112 -0.77459667 -0.85156724 ... -0.9486833  -0.79492904
  -0.79549848]
 ...
 [ 0.75456956  0.77459667  0.53031625 ...  0.9486833   0.32733629
   0.33795544]
 [ 1.15807816  1.161895    1.07187738 ...  0.9486833   0.94651716
   0.96086229]
 [ 1.57300683  1.54919334  2.05571343 ...  1.8973666   2.03008369
   2.04503561]]
[[-1.69841555]
 [-1.69841555]
 [-1.69841555]
 ...
 [ 1.69841555]
 [ 1.69841555]
 [ 1.69841555]]


In [6]:
n_examples = y.shape[0]
expectNLL = lambda y, rv_y: -rv_y.log_prob(y)/n_examples

In [7]:
l2_reg = tf.keras.regularizers.L2(l2=0.05)

input = tf.keras.layers.Input(16)
h1 = tf.keras.layers.Dense(50, activation=tf.keras.activations.relu, name="hidden1")(input)
h2 = tf.keras.layers.Dense(50, activation=tf.keras.activations.relu, name="hidden2")(h1)
h3 = tf.keras.layers.Dense(20, activation=tf.keras.activations.relu, name="hidden3")(h2)
m2 = tf.keras.layers.Dense(1, name="mu", kernel_regularizer=l2_reg,bias_regularizer=l2_reg)(h3)
s2 = tf.keras.layers.Dense(1,activation=tf.keras.activations.exponential,kernel_regularizer=l2_reg, bias_regularizer=l2_reg, name="sigma_L2_reg")(h3)
p2 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t[0], scale=t[1]), name="output")([m2, s2])
model = tf.keras.Model(inputs=input, outputs=p2, name="NavalModel")

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=expectNLL)
print(model.summary())

Model: "NavalModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
hidden1 (Dense)                 (None, 50)           850         input_1[0][0]                    
__________________________________________________________________________________________________
hidden2 (Dense)                 (None, 50)           2550        hidden1[0][0]                    
__________________________________________________________________________________________________
hidden3 (Dense)                 (None, 20)           1020        hidden2[0][0]                    
_________________________________________________________________________________________

2023-02-06 23:40:08.870517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-06 23:40:08.871865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-06 23:40:08.871901: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your pl

In [8]:
model.fit(X, y, epochs=100, verbose=True)

2023-02-06 23:40:09.062658: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
373/373 [==============================] - 1s 2ms/step - loss: 0.0059
Epoch 2/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1893e-04
Epoch 3/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1893e-04
Epoch 4/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1894e-04
Epoch 5/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1896e-04
Epoch 6/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1895e-04
Epoch 7/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1896e-04
Epoch 8/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1895e-04
Epoch 9/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1896e-04
Epoch 10/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1896e-04
Epoch 11/100
373/373 [==============================] - 0s 1ms/step - loss: 1.1898e-04
Epoch 12/100
373/373 [==============================] - 

In [9]:
yhat = model.predict(X)
yhat = scaler_y.inverse_transform(yhat)
print(yhat)

[[0.9704076 ]
 [0.9771641 ]
 [0.96479225]
 ...
 [0.9456068 ]
 [0.9889828 ]
 [0.9506123 ]]


In [10]:
from sklearn.metrics import mean_squared_error

print('RMSE:', np.sqrt(mean_squared_error(y, yhat)))

RMSE: 1.3967553122560445


In [11]:
# Laplace Approximation copied from Model Code notebook

class LaplaceApproximation(tf.keras.Model):

    def __init__(self,model):
        super().__init__()
        self.model = model
        self.mean = None
        self.std = None
        self.lam = None

    def fit(self,dataset,training=False):
        """
        Fit the Laplace approximation for a model. Setting the mean of the Laplace approximation to the weights of the model 
            and, using the diagonal of the Fisher matrix, set the standard deviation of the Laplace approximation.
        :param model: Model whose Fisher matrix is to be computed.
        :param dataset: Dataset which the model has been trained on, but which will not be seen in the future. 
            Formatted as (inputs, labels).
        :param samples: Number of samples to take from the dataset. More samples
            gives a better approximation of the true variance.
        """
        inputs, labels = dataset
        trainable_weights = self.model.trainable_weights
        variance = [tf.zeros_like(tensor) for tensor in trainable_weights]
        
        pb = tf.keras.utils.Progbar(inputs.shape[0])

        for i in range(inputs.shape[0]):
            pb.update(i)
            index = i#np.random.randint(len(inputs))
            # Select an element from the dataset.
            data = inputs[index]
            target = labels[index]

            # When extracting from the array we lost a dimension so put it back.
            data = tf.expand_dims(data, axis=0)

            # Collect gradients.
            with tf.GradientTape() as tape:
                output = self.model(data,training=training)
                log_likelihood = output.log_prob(target)

            gradients = tape.gradient(log_likelihood, trainable_weights)

            # If the model has converged, we can assume that the current weights
            # are the mean, and each gradient we see is a deviation. The variance is
            # the average of the square of this deviation.
            variance = [var + (grad ** 2) for var, grad in zip(variance, gradients)]

        self.fisher_diagonal = [tensor.numpy() for tensor in variance]
        #self.fisher_diagonal = [tensor.numpy() / samples * inputs.shape[0] for tensor in variance]
        
        mins = [np.min(f) for f in self.fisher_diagonal]
        for i,x in enumerate(mins):
          if i==0 or min > x:
            min = x
        print(min)
        #print('FISHER')
        #print(fisher_diagonal)
        self.mean = self.model.get_weights()       
        

    def reset_weights(self):
        if self.mean:
            self.model.set_weights(self.mean)

    def set_std(self, weight_decay):
      self.std = [np.reciprocal(np.sqrt(x+weight_decay)) for x in self.fisher_diagonal]

    def call(self, inputs, training=False):
        e = [np.random.standard_normal(x.shape) for x in self.mean]
        sample = [m+s*e for m, s, e in zip(self.mean,self.std,e)]
        self.model.set_weights(sample)
        return self.model(inputs,training=training)

In [12]:
la_model = LaplaceApproximation(model)

In [13]:
la_model.fit([X,y])

11932/11934 [============================>.] - ETA: 0s0.0


In [14]:
la_model.set_std(100)

In [15]:
yhat2 = la_model.predict(X)
yhat2 = scaler_y.inverse_transform(yhat2)
print(yhat2)

[[0.9564295 ]
 [0.9748274 ]
 [0.9666824 ]
 ...
 [0.97935134]
 [1.0003133 ]
 [0.9823706 ]]


In [16]:
print('RMSE:', np.sqrt(mean_squared_error(y, yhat)))

RMSE: 1.3967553122560445


In [17]:
y_dist = la_model(X)
means = y_dist.mean().numpy()
print('Epistemic Uncertainty', means.var())

Epistemic Uncertainty 0.0024254636


# Ionospheric Dataset

In [18]:
ion_data = pd.read_csv('ionosphere.csv', header=None)
print(ion_data.info)

n_examples = y.shape[0]
expectNLL = lambda y, rv_y: -rv_y.log_prob(y)/n_examples

<bound method DataFrame.info of      0   1        2        3        4        5        6        7        8   \
0     1   0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1     1   0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2     1   0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3     1   0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4     1   0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   
..   ..  ..      ...      ...      ...      ...      ...      ...      ...   
346   1   0  0.83508  0.08298  0.73739 -0.14706  0.84349 -0.05567  0.90441   
347   1   0  0.95113  0.00419  0.95183 -0.02723  0.93438 -0.01920  0.94590   
348   1   0  0.94701 -0.00034  0.93207 -0.03227  0.95177 -0.03431  0.95584   
349   1   0  0.90608 -0.01657  0.98122 -0.01989  0.95691 -0.03646  0.85746   
350   1   0  0.84710  0.13533  0.73638 -0.06151  0.87873  0.08260  0.88928   

          9   ...       25     

In [19]:
"""
# One hot, label is last two columns
def one_hot(cat, hot):
    if cat == hot:
        return int(1)
    else:
        return int(0)
    
ion_data[35] = ion_data[34].apply(one_hot, hot='g')
ion_data[36] = ion_data[34].apply(one_hot, hot='b')
"""

def ordinal_encoder(category):
    dict = {'g': 0, 'b': 1}
    return dict[category]

print('g class:', ordinal_encoder('g'))
print('b class:', ordinal_encoder('b'))
ion_data[34] = ion_data[34].apply(ordinal_encoder)
print(ion_data[34].to_numpy())

g class: 0
b class: 1
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
# split x and y
x = ion_data.iloc[:,0:33].to_numpy()
y = ion_data.iloc[:,34].to_numpy().reshape(-1,1)

# Scaling
scaler = StandardScaler()
x = scaler.fit_transform(x)
print(x.shape)
print(y.shape)

(351, 33)
(351, 1)


In [21]:
input2 = tf.keras.layers.Input(33)
h21 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden1")(input2)
h22 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden2")(h21)
h23 = tf.keras.layers.Dense(10, activation=tf.keras.activations.relu, name="hidden3")(h22)
p_val = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="p_val")(h23)
out2 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(probs=t,  dtype=tf.float64), name="output")(p_val)

model2 = tf.keras.Model(inputs=input2, outputs=out2, name="IonosphereNNMLEwDistr")

model2.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=expectNLL)
print(model2.summary())

Model: "IonosphereNNMLEwDistr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
hidden1 (Dense)              (None, 10)                340       
_________________________________________________________________
hidden2 (Dense)              (None, 10)                110       
_________________________________________________________________
hidden3 (Dense)              (None, 10)                110       
_________________________________________________________________
p_val (Dense)                (None, 1)                 11        
_________________________________________________________________
output (DistributionLambda)  multiple                  0         
Total params: 571
Trainable params: 571
Non-trainable params: 0
_______________________________________________

In [22]:
model2.fit(x, y, epochs=50, verbose=False)

In [23]:
from sklearn.metrics import classification_report

yhat3 = model2.predict(x)

def labelmaker(prob):
    if prob > 0.4:
        return 1
    else:
        return 0

yhat3 = np.apply_along_axis(labelmaker, 1, yhat3)
print(classification_report(y.flatten(), yhat3.flatten()))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       225
           1       0.92      0.90      0.91       126

    accuracy                           0.93       351
   macro avg       0.93      0.93      0.93       351
weighted avg       0.93      0.93      0.93       351



In [24]:
la_model2 = LaplaceApproximation(model2)
la_model2.fit([x,y])
la_model2.set_std(100)

348/351 [============================>.] - ETA: 0s0.0


In [25]:
yhat3 = la_model2.predict(x)

def labelmaker(prob):
    if prob > 0.4:
        return 1
    else:
        return 0

yhat3 = np.apply_along_axis(labelmaker, 1, yhat3)
print(classification_report(y.flatten(), yhat3.flatten()))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       225
           1       0.85      0.87      0.86       126

    accuracy                           0.90       351
   macro avg       0.89      0.89      0.89       351
weighted avg       0.90      0.90      0.90       351



In [26]:
y_dist2 = la_model2(x)

means = y_dist2.mean().numpy()

print('Epistemic Uncertainty', means.var())

Epistemic Uncertainty 0.17253219
